In [217]:
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols

import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#hiding warning messages
import warnings
warnings.filterwarnings("ignore")

#Reading in Molecular Properties CSV
data = pd.read_csv('Downsampled_Tox21_Full')
#data = data.astype(float, errors = 'ignore')
data.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,Activity Summary,PUBCHEM_CID,...,InChI,XLogP,ExactMass,TPSA,HBondDonorCount,HBondAcceptorCount,RotatableBondCount,HeavyAtomCount,Complexity,target
0,-1.088679,-1.012653,-0.424966,0.696715,-0.251154,0.175948,0.395954,-0.002846,active antagonist,1807.0,...,InChI=1S/C4H6BrNO4/c5-4(6(7)8)1-9-3-10-2-4/h1-3H2,0.3,210.948020,64.3,0.0,4.0,0.0,10.0,139.0,1
1,-0.962211,0.055747,-0.737993,-0.252623,0.336321,-0.107655,0.534785,-0.122667,active antagonist,2453.0,...,InChI=1S/C9H5Br2NO/c10-6-4-7(11)9(13)8-5(6)2-1...,3.2,302.871740,33.1,1.0,2.0,0.0,13.0,191.0,1
2,1.304541,0.354035,-0.929372,0.667715,0.164955,0.039851,0.033271,0.042059,active antagonist,2662.0,...,InChI=1S/C17H14F3N3O2S/c1-11-2-4-12(5-3-11)15-...,3.4,381.075882,86.4,1.0,7.0,3.0,26.0,577.0,1
3,-1.190718,0.025848,-0.655560,-0.202411,0.475440,0.042973,0.090642,-0.006175,active antagonist,2722.0,...,InChI=1S/C9H5Cl2NO/c10-6-4-7(11)9(13)8-5(6)2-1...,3.5,212.974819,33.1,1.0,2.0,0.0,13.0,191.0,1
4,-0.305714,1.563062,-0.637116,-0.372404,-0.206055,-0.265096,0.096813,0.226429,active antagonist,2812.0,...,InChI=1S/C22H17ClN2/c23-21-14-8-7-13-20(21)22(...,5.0,344.108026,17.8,0.0,1.0,4.0,25.0,396.0,1


# MACCS Keys

In [218]:
#Making list of isomericSMILES identifiers
iSMILES = data['IsomericSMILES']
iSMILES[0]

'C1C(COCO1)([N+](=O)[O-])Br'

In [219]:
#Making list of molecules from iSMILES
i = 0
ms = []
while (i < len(iSMILES)):
    molecule = Chem.MolFromSmiles(iSMILES[i])
    ms.append(molecule)
    i = i + 1

In [220]:
#Getting MACCS fingerprints list
#Turning fingerprints to bit strings
maccs_fps = [MACCSkeys.GenMACCSKeys(x).ToBitString()[1:] for x in ms ]

# Topological Fingerprint (Daylight Analogue)

In [221]:
from rdkit.Chem import rdmolops

#getting list of topological fingerprints
top_fps = [rdmolops.RDKFingerprint(x, fpSize=2048, minPath=1, maxPath=7).ToBitString() for x in ms]

# Morgan Fingerprint (ECFP)

In [222]:
from rdkit.Chem import AllChem

#getting morgan ecfp fingerprint
ecfp_fps = [AllChem.GetMorganFingerprintAsBitVect(x,4,nBits=1024).ToBitString() for x in ms]

# Morgan Fingerprint (FCFP)

In [223]:
fcfp_fps = [AllChem.GetMorganFingerprintAsBitVect(x,4,nBits=1024,useFeatures=True).ToBitString() for x in ms]

# PubChem FP Decoding

In [224]:
#Getting CID list
CID_list = data['PUBCHEM_CID']

#Making them integers (removing decimals)
CID_list = [int(i) for i in CID_list]

list1 = CID_list[:600]
list2 = CID_list[600:]

#Changing list to str, dropping start/end brackets, removing spaces
str1 = (str(list1)[1:-1])
str1 = str1.replace(' ', '')
str2 = (str(list2)[1:-1])
str2 = str2.replace(' ', '')


In [225]:
#Getting the Pubchem Fingerprints for each CID

#opening and reading the 1URLs
url1 = ('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/' + str1 + '/property/Fingerprint2D/TXT')
html1 = urlopen(url1) 
soup1 = BeautifulSoup(html1, 'lxml')
url2 = ('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/' + str2 + '/property/Fingerprint2D/TXT')
html2 = urlopen(url2) 
soup2 = BeautifulSoup(html2, 'lxml')

pub_fp1 = soup1.get_text()
pub_fp2 = soup2.get_text()

#total pub_fp string
pub_fp = pub_fp1 + pub_fp2 

#pub_fp string to pub_fp list
pub_fp = pub_fp.split()

In [226]:
#Decoding Fingerprints
from base64 import b64decode

def PCFP_BitString(pcfp_base64) :

    pcfp_bitstring = "".join( ["{:08b}".format(x) for x in b64decode( pcfp_base64 )] )[32:913]
    return pcfp_bitstring

i = 0
pub_fp_decoded = []
while (i < len(pub_fp)):
    fp = PCFP_BitString(pub_fp[i])
    pub_fp_decoded.append(fp)
    i = i + 1

# FinalDFTox21

In [227]:
fps_df = pd.DataFrame()
fps_df['Name'] = data['PUBCHEM_CID']
fps_df['MACCS'] = maccs_fps
fps_df['Topological'] = top_fps
fps_df['Morgan ECFP'] = ecfp_fps
fps_df['Morgan FCFP'] = fcfp_fps
fps_df['Pubchem FP'] = pub_fp_decoded
fps_df['Activity'] = data['target']
fps_df.head()

,Name,MACCS,Topological,Morgan ECFP,Morgan FCFP,Pubchem FP,Activity
0,1807.0,0000000000000000000000010001000000000000000001...,0000010000000000000000000000000010001000000000...,0000000000000000000000000000000100001000000000...,1010000010000000100000000010000000000000000000...,1000000001100010001110000000000000000000000100...,1
1,2453.0,0000000000000000000000000000000000000000000001...,1000010000010000000001000100100000100000000010...,0000000000000000000100000000000000000000000000...,0001101010000000000000000000000000000001000000...,1000000001110010001000000000000000000000000110...,1
2,2662.0,0000000000000000000000000000000110000000010000...,1111010010100000101110100011001101001001100100...,0000100000000000000000000000000001000000000000...,1110101011000000000000000000000000000000000000...,1100000001111011001100011000000001000000000000...,1
3,2722.0,0000000000000000000000000000000000000000000000...,1000010010010000100010000110000000100000010010...,0000000000000000000100000000000000000000000000...,0001101010000000000000000000000000000001000000...,1000000001110010001000000000000000000110000000...,1
4,2812.0,0000000000000000000000000000000000000000000000...,0001011100000101101010001001010110110000000000...,0100000010000000000000100000000000000000000000...,1000101010001100001000000000000000000000000000...,1110000001111011000000000000000000000100000000...,1


In [228]:
#Writing to csv
fps_df.to_csv('Fingerprints Tox21', index = False)


# Formatting - MACCS

In [229]:
#making list of maccs1, maccs2, etc.
i = 0
maccs_names = ['Name']
while (i < len(maccs_fps[0])):
    string = "MACCS" + str(i + 1)
    maccs_names.append(string)
    i = i + 1
    
#Putting quotes around each CID 
#These are strings, should I change to int
i = 0
CID_list = fps_df['Name'].tolist()
CID_str = []
while (i < len(CID_list)):
    string = "\'" + str(CID_list[i]) + "\'"
    CID_str.append(string)
    i = i + 1
    
#Splitting each bitstring fp into individual bits, and putting them in a df by row
first_row = []
first_row.append(CID_list[0])
first_row.extend([int(a) for a in str(maccs_fps[0])])
maccs_df = pd.DataFrame(first_row).T
x = 1
row = []
while (x < len(maccs_fps)):
    bit_row1 = [int(y) for y in str(maccs_fps[x])]
    row.append(CID_list[x])
    row.extend(bit_row1)
    row1df = pd.DataFrame(row).T
    maccs_df = maccs_df.append(row1df)
    row.clear()   
    x = x + 1

#Reindexing df, MACCS1, MACCS2, etc as column headers
maccs_df.columns = [maccs_names]

#removing decimals
maccs_df = maccs_df.astype(int)

In [230]:
#Writing fingerpring information
maccs_df.to_csv('maccs_fps.csv', index = False)

In [231]:
#Adding activity_score/Making input train data
#making list of maccs1, maccs2, etc.
i = 0
maccs_names = ['Name', 'Activity']
while (i < len(maccs_fps[0])):
    string = "MACCS" + str(i + 1)
    maccs_names.append(string)
    i = i + 1

i = 0
CID_list = fps_df['Name'].tolist()
act_list = fps_df['Activity'].tolist()

first_row = []
first_row.append(CID_list[0])
first_row.append(act_list[0])
first_row.extend([int(a) for a in str(maccs_fps[0])])
input_train_maccs = pd.DataFrame(first_row).T
x = 1
row = []
while (x < len(maccs_fps)):
    bit_row1 = [int(y) for y in str(maccs_fps[x])]
    row.append(CID_list[x])
    row.append(act_list[x])
    row.extend(bit_row1)
    row1df = pd.DataFrame(row).T
    input_train_maccs = input_train_maccs.append(row1df)
    row.clear()   
    x = x + 1
    
#Reindexing df, MACCS1, MACCS2, etc as column headers
input_train_maccs.columns = [maccs_names]

#removing decimals
input_train_maccs = input_train_maccs.astype(int)

In [233]:
#Writing TRAIN MACCS DATA(to TAB DELIMTED FILE?)
input_train_maccs.to_csv('input_train_maccs.csv', index = False)

# Formatting - Topological Fingerprint 

In [234]:
#making list of names
i = 0
top_names = ['Name']
while (i < len(top_fps[0])):
    string = "TOP" + str(i + 1)
    top_names.append(string)
    i = i + 1

#Splitting each bitstring fp into individual bits, and putting them in a df by row
first_row = []
first_row.append(CID_list[0])
first_row.extend([int(a) for a in str(top_fps[0])])
top_df = pd.DataFrame(first_row).T
x = 1
row = []
while (x < len(top_fps)):
    bit_row1 = [int(y) for y in str(top_fps[x])]
    row.append(CID_list[x])
    row.extend(bit_row1)
    row1df = pd.DataFrame(row).T
    top_df = top_df.append(row1df)
    row.clear()   
    x = x + 1
top_df.shape 


#Reindexing df, MACCS1, MACCS2, etc as column headers
top_df.columns = [top_names]

#removing decimals
top_df = top_df.astype(int)


In [235]:
#Writing to csv
top_df.to_csv('top_fps.csv', index = False)

In [236]:
#Adding activity_score/Making input train data
#making list of names etc.
i = 0
top_names = ['Name', 'Activity']
while (i < len(top_fps[0])):
    string = "TOP" + str(i + 1)
    top_names.append(string)
    i = i + 1

first_row = []
first_row.append(CID_list[0])
first_row.append(act_list[0])
first_row.extend([int(a) for a in str(top_fps[0])])
input_train_top = pd.DataFrame(first_row).T
x = 1
row = []
while (x < len(top_fps)):
    bit_row1 = [int(y) for y in str(top_fps[x])]
    row.append(CID_list[x])
    row.append(act_list[x])
    row.extend(bit_row1)
    row1df = pd.DataFrame(row).T
    input_train_top = input_train_top.append(row1df)
    row.clear()   
    x = x + 1
    
#Reindexing df, MACCS1, MACCS2, etc as column headers
input_train_top.columns = [top_names]

#removing decimals
input_train_top = input_train_top.astype(int)

In [237]:
#Writing TRAIN Topological DATA(to TAB DELIMTED FILE?)
input_train_top.to_csv('input_train_top.csv', index = False)

# Formatting - Morgan (ecfp)

In [238]:
#making list of names
i = 0
ecfp_names = ['Name']
while (i < len(ecfp_fps[0])):
    string = "ECFP" + str(i + 1)
    ecfp_names.append(string)
    i = i + 1

#Splitting each bitstring fp into individual bits, and putting them in a df by row
first_row = []
first_row.append(CID_list[0])
first_row.extend([int(a) for a in str(ecfp_fps[0])])
ecfp_df = pd.DataFrame(first_row).T
x = 1
row = []
while (x < len(ecfp_fps)):
    bit_row1 = [int(y) for y in str(ecfp_fps[x])]
    row.append(CID_list[x])
    row.extend(bit_row1)
    row1df = pd.DataFrame(row).T
    ecfp_df = ecfp_df.append(row1df)
    row.clear()   
    x = x + 1
ecfp_df.shape 


#Reindexing df, MACCS1, MACCS2, etc as column headers
ecfp_df.columns = [ecfp_names]

#removing decimals
ecfp_df = ecfp_df.astype(int)


In [239]:
#Writing to csv
ecfp_df.to_csv('ecfp_fps.csv', index = False)

In [240]:
#Adding activity_score/Making input train data
#making list of names etc.
i = 0
ecfp_names = ['Name', 'Activity']
while (i < len(ecfp_fps[0])):
    string = "ECFP" + str(i + 1)
    ecfp_names.append(string)
    i = i + 1

first_row = []
first_row.append(CID_list[0])
first_row.append(act_list[0])
first_row.extend([int(a) for a in str(ecfp_fps[0])])
input_train_ecfp = pd.DataFrame(first_row).T
x = 1
row = []
while (x < len(ecfp_fps)):
    bit_row1 = [int(y) for y in str(ecfp_fps[x])]
    row.append(CID_list[x])
    row.append(act_list[x])
    row.extend(bit_row1)
    row1df = pd.DataFrame(row).T
    input_train_ecfp = input_train_ecfp.append(row1df)
    row.clear()   
    x = x + 1
    
#Reindexing df, namesetc as column headers
input_train_ecfp.columns = [ecfp_names]

#removing decimals
input_train_ecfp = input_train_ecfp.astype(int)

In [241]:
#Writing TRAIN DATA(to TAB DELIMTED FILE?)
input_train_ecfp.to_csv('input_train_ecfp.csv', index = False)

# Formatting - Morgan (fcfp)

In [242]:
#making list of names
i = 0
fcfp_names = ['Name']
while (i < len(fcfp_fps[0])):
    string = "FCFP" + str(i + 1)
    fcfp_names.append(string)
    i = i + 1

#Splitting each bitstring fp into individual bits, and putting them in a df by row
first_row = []
first_row.append(CID_list[0])
first_row.extend([int(a) for a in str(fcfp_fps[0])])
fcfp_df = pd.DataFrame(first_row).T
x = 1
row = []
while (x < len(fcfp_fps)):
    bit_row1 = [int(y) for y in str(fcfp_fps[x])]
    row.append(CID_list[x])
    row.extend(bit_row1)
    row1df = pd.DataFrame(row).T
    fcfp_df = fcfp_df.append(row1df)
    row.clear()   
    x = x + 1
fcfp_df.shape 


#Reindexing df, MACCS1, MACCS2, etc as column headers
fcfp_df.columns = [fcfp_names]

#removing decimals
fcfp_df = fcfp_df.astype(int)


In [243]:
#Writing to csv
fcfp_df.to_csv('fcfp_fps.csv', index = False)

In [244]:
#Adding activity_score/Making input train data
#making list of names etc.
i = 0
fcfp_names = ['Name', 'Activity']
while (i < len(fcfp_fps[0])):
    string = "FCFP" + str(i + 1)
    fcfp_names.append(string)
    i = i + 1

#Splitting each bitstring fp into individual bits, and putting them in a df by row
first_row = []
first_row.append(CID_list[0])
first_row.append(act_list[0])
first_row.extend([int(a) for a in str(fcfp_fps[0])])
input_train_fcfp = pd.DataFrame(first_row).T
x = 1
row = []
while (x < len(fcfp_fps)):
    bit_row1 = [int(y) for y in str(fcfp_fps[x])]
    row.append(CID_list[x])
    row.append(act_list[x])
    row.extend(bit_row1)
    row1df = pd.DataFrame(row).T
    input_train_fcfp = input_train_fcfp.append(row1df)
    row.clear()   
    x = x + 1

    
#Reindexing df, namesetc as column headers
input_train_fcfp.columns = [fcfp_names]

#removing decimals
input_train_fcfp = input_train_fcfp.astype(int)

In [245]:
#Writing TRAINING DATA to csv
input_train_fcfp.to_csv('input_train_fcfp.csv', index = False)

# Formatting - PubchemFP

In [246]:
#making list of names
i = 0
pub_fp_names = ['Name']
while (i < len(pub_fp_decoded[0])):
    string = "PubFP" + str(i + 1)
    pub_fp_names.append(string)
    i = i + 1

#Splitting each bitstring fp into individual bits, and putting them in a df by row
first_row = []
first_row.append(CID_list[0])
first_row.extend([int(a) for a in str(pub_fp_decoded[0])])
pub_fp_df = pd.DataFrame(first_row).T
x = 1
row = []
while (x < len(pub_fp_decoded)):
    bit_row1 = [int(y) for y in str(pub_fp_decoded[x])]
    row.append(CID_list[x])
    row.extend(bit_row1)
    row1df = pd.DataFrame(row).T
    pub_fp_df = pub_fp_df.append(row1df)
    row.clear()   
    x = x + 1
pub_fp_df.shape 


#Reindexing df, MACCS1, MACCS2, etc as column headers
pub_fp_df.columns = [pub_fp_names]

#removing decimals
pub_fp_df = pub_fp_df.astype(int)

In [247]:
#Writing to csv
pub_fp_df.to_csv('pub_fps.csv', index = False)

In [248]:
#Adding activity_score/Making input train data
#making list of names etc.
i = 0
pub_fp_names = ['Name', 'Activity']
while (i < len(pub_fp_decoded[0])):
    string = "PubFP" + str(i + 1)
    pub_fp_names.append(string)
    i = i + 1

first_row = []
first_row.append(CID_list[0])
first_row.append(act_list[0])
first_row.extend([int(a) for a in str(pub_fp_decoded[0])])
input_train_pub = pd.DataFrame(first_row).T
x = 1
row = []
while (x < len(pub_fp_decoded)):
    bit_row1 = [int(y) for y in str(pub_fp_decoded[x])]
    row.append(CID_list[x])
    row.append(act_list[x])
    row.extend(bit_row1)
    row1df = pd.DataFrame(row).T
    input_train_pub = input_train_pub.append(row1df)
    row.clear()   
    x = x + 1

    
#Reindexing df, namesetc as column headers
input_train_pub.columns = [pub_fp_names]

#removing decimals
input_train_pub = input_train_pub.astype(int)

In [249]:
#Writing TRAINING DATA to csv
input_train_pub.to_csv('input_train_pub.csv', index = False)

# Repeating Process for Test Set

In [250]:

data = pd.read_csv('Test_Tox21')
#data = data.astype(float, errors = 'ignore')
data.head()

# MACCS Keys

#Making list of isomericSMILES identifiers
iSMILES = data['IsomericSMILES']
iSMILES[0]

#Making list of molecules from iSMILES
i = 0
ms = []
while (i < len(iSMILES)):
    molecule = Chem.MolFromSmiles(iSMILES[i])
    ms.append(molecule)
    i = i + 1

#Getting MACCS fingerprints list
#Turning fingerprints to bit strings
maccs_fps = [MACCSkeys.GenMACCSKeys(x).ToBitString()[1:] for x in ms ]

# Topological Fingerprint (Daylight Analogue)

from rdkit.Chem import rdmolops

#getting list of topological fingerprints
top_fps = [rdmolops.RDKFingerprint(x, fpSize=2048, minPath=1, maxPath=7).ToBitString() for x in ms]

# Morgan Fingerprint (ECFP)

from rdkit.Chem import AllChem

#getting morgan ecfp fingerprint
ecfp_fps = [AllChem.GetMorganFingerprintAsBitVect(x,4,nBits=1024).ToBitString() for x in ms]

# Morgan Fingerprint (FCFP)

fcfp_fps = [AllChem.GetMorganFingerprintAsBitVect(x,4,nBits=1024,useFeatures=True).ToBitString() for x in ms]

# PubChem FP Decoding

#Getting CID list
CID_list = data['PUBCHEM_CID']

#Making them integers (removing decimals)
CID_list = [int(i) for i in CID_list]

list1 = CID_list[:600]
list2 = CID_list[600:]

#Changing list to str, dropping start/end brackets, removing spaces
str1 = (str(list1)[1:-1])
str1 = str1.replace(' ', '')
str2 = (str(list2)[1:-1])
str2 = str2.replace(' ', '')


#Getting the Pubchem Fingerprints for each CID

#opening and reading the 1URLs
url1 = ('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/' + str1 + '/property/Fingerprint2D/TXT')
html1 = urlopen(url1) 
soup1 = BeautifulSoup(html1, 'lxml')
url2 = ('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/' + str2 + '/property/Fingerprint2D/TXT')
html2 = urlopen(url2) 
soup2 = BeautifulSoup(html2, 'lxml')

pub_fp1 = soup1.get_text()
pub_fp2 = soup2.get_text()

#total pub_fp string
pub_fp = pub_fp1 + pub_fp2 

#pub_fp string to pub_fp list
pub_fp = pub_fp.split()

#Decoding Fingerprints
from base64 import b64decode

def PCFP_BitString(pcfp_base64) :

    pcfp_bitstring = "".join( ["{:08b}".format(x) for x in b64decode( pcfp_base64 )] )[32:913]
    return pcfp_bitstring

i = 0
pub_fp_decoded = []
while (i < len(pub_fp)):
    fp = PCFP_BitString(pub_fp[i])
    pub_fp_decoded.append(fp)
    i = i + 1

# FinalDFTox21

fps_df = pd.DataFrame()
fps_df['Name'] = data['PUBCHEM_CID']
fps_df['MACCS'] = maccs_fps
fps_df['Topological'] = top_fps
fps_df['Morgan ECFP'] = ecfp_fps
fps_df['Morgan FCFP'] = fcfp_fps
fps_df['Pubchem FP'] = pub_fp_decoded
fps_df['Activity'] = data['target']
fps_df.head()

#Writing to csv
fps_df.to_csv('Fingerprints Tox21 Test', index = False)
















# Formatting - MACCS

#Adding activity_score/Making input train data
#making list of maccs1, maccs2, etc.
i = 0
maccs_names = ['Name', 'Activity']
while (i < len(maccs_fps[0])):
    string = "MACCS" + str(i + 1)
    maccs_names.append(string)
    i = i + 1

i = 0
CID_list = fps_df['Name'].tolist()
act_list = fps_df['Activity'].tolist()

first_row = []
first_row.append(CID_list[0])
first_row.append(act_list[0])
first_row.extend([int(a) for a in str(maccs_fps[0])])
input_train_maccs = pd.DataFrame(first_row).T
x = 1
row = []
while (x < len(maccs_fps)):
    bit_row1 = [int(y) for y in str(maccs_fps[x])]
    row.append(CID_list[x])
    row.append(act_list[x])
    row.extend(bit_row1)
    row1df = pd.DataFrame(row).T
    input_train_maccs = input_train_maccs.append(row1df)
    row.clear()   
    x = x + 1
    
#Reindexing df, MACCS1, MACCS2, etc as column headers
input_train_maccs.columns = [maccs_names]

#removing decimals
input_train_maccs = input_train_maccs.astype(int)

#Writing TRAIN MACCS DATA(to TAB DELIMTED FILE?)
input_train_maccs.to_csv('input_test_maccs.csv', index = False)

# Formatting - Topological Fingerprint 


#Adding activity_score/Making input train data
#making list of names etc.
i = 0
top_names = ['Name', 'Activity']
while (i < len(top_fps[0])):
    string = "TOP" + str(i + 1)
    top_names.append(string)
    i = i + 1

first_row = []
first_row.append(CID_list[0])
first_row.append(act_list[0])
first_row.extend([int(a) for a in str(top_fps[0])])
input_train_top = pd.DataFrame(first_row).T
x = 1
row = []
while (x < len(top_fps)):
    bit_row1 = [int(y) for y in str(top_fps[x])]
    row.append(CID_list[x])
    row.append(act_list[x])
    row.extend(bit_row1)
    row1df = pd.DataFrame(row).T
    input_train_top = input_train_top.append(row1df)
    row.clear()   
    x = x + 1
    
#Reindexing df, MACCS1, MACCS2, etc as column headers
input_train_top.columns = [top_names]

#removing decimals
input_train_top = input_train_top.astype(int)

#Writing TRAIN Topological DATA(to TAB DELIMTED FILE?)
input_train_top.to_csv('input_test_top.csv', index = False)


# Formatting - Morgan (ecfp)

#Adding activity_score/Making input train data
#making list of names etc.
i = 0
ecfp_names = ['Name', 'Activity']
while (i < len(ecfp_fps[0])):
    string = "ECFP" + str(i + 1)
    ecfp_names.append(string)
    i = i + 1

first_row = []
first_row.append(CID_list[0])
first_row.append(act_list[0])
first_row.extend([int(a) for a in str(ecfp_fps[0])])
input_train_ecfp = pd.DataFrame(first_row).T
x = 1
row = []
while (x < len(ecfp_fps)):
    bit_row1 = [int(y) for y in str(ecfp_fps[x])]
    row.append(CID_list[x])
    row.append(act_list[x])
    row.extend(bit_row1)
    row1df = pd.DataFrame(row).T
    input_train_ecfp = input_train_ecfp.append(row1df)
    row.clear()   
    x = x + 1
    
#Reindexing df, namesetc as column headers
input_train_ecfp.columns = [ecfp_names]

#removing decimals
input_train_ecfp = input_train_ecfp.astype(int)

#Writing TRAIN DATA(to TAB DELIMTED FILE?)
input_train_ecfp.to_csv('input_test_ecfp.csv', index = False)



# Formatting - Morgan (fcfp)

#Adding activity_score/Making input train data
#making list of names etc.
i = 0
fcfp_names = ['Name', 'Activity']
while (i < len(fcfp_fps[0])):
    string = "FCFP" + str(i + 1)
    fcfp_names.append(string)
    i = i + 1

#Splitting each bitstring fp into individual bits, and putting them in a df by row
first_row = []
first_row.append(CID_list[0])
first_row.append(act_list[0])
first_row.extend([int(a) for a in str(fcfp_fps[0])])
input_train_fcfp = pd.DataFrame(first_row).T
x = 1
row = []
while (x < len(fcfp_fps)):
    bit_row1 = [int(y) for y in str(fcfp_fps[x])]
    row.append(CID_list[x])
    row.append(act_list[x])
    row.extend(bit_row1)
    row1df = pd.DataFrame(row).T
    input_train_fcfp = input_train_fcfp.append(row1df)
    row.clear()   
    x = x + 1

    
#Reindexing df, namesetc as column headers
input_train_fcfp.columns = [fcfp_names]

#removing decimals
input_train_fcfp = input_train_fcfp.astype(int)

#Writing TRAINING DATA to csv
input_train_fcfp.to_csv('input_test_fcfp.csv', index = False)

# Formatting - PubchemFP

#Adding activity_score/Making input train data
#making list of names etc.
i = 0
pub_fp_names = ['Name', 'Activity']
while (i < len(pub_fp_decoded[0])):
    string = "PubFP" + str(i + 1)
    pub_fp_names.append(string)
    i = i + 1

first_row = []
first_row.append(CID_list[0])
first_row.append(act_list[0])
first_row.extend([int(a) for a in str(pub_fp_decoded[0])])
input_train_pub = pd.DataFrame(first_row).T
x = 1
row = []
while (x < len(pub_fp_decoded)):
    bit_row1 = [int(y) for y in str(pub_fp_decoded[x])]
    row.append(CID_list[x])
    row.append(act_list[x])
    row.extend(bit_row1)
    row1df = pd.DataFrame(row).T
    input_train_pub = input_train_pub.append(row1df)
    row.clear()   
    x = x + 1

    
#Reindexing df, namesetc as column headers
input_train_pub.columns = [pub_fp_names]

#removing decimals
input_train_pub = input_train_pub.astype(int)

#Writing TRAINING DATA to csv
input_train_pub.to_csv('input_test_pub.csv', index = False)

